# Training the model

Project Data Mining - Mata kuliah Data Mining Semester 6 (Universitas Indraprastra PGRI)

Notebook ini digunakan untuk membuat model pada Project [**Aplikasi Digit Classifier Tulisan Tangan dengan TensorFlow Lite (Menggunakan Teknik CNN)**](https://www.youtube.com/watch?v=ZATebqtGHrM).

Kelompok 4:
- Aji Rifqi N (202143502395)
- Alfareza K Deva M (202143502334)
- Reza Khareisa Putra (202143502344)
- Syarla Capriliani (202143502344)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1hfeLdKj1VVkIYxuCv5NPkiTr1UolVHR1">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>

## Import dependencies

Import TensorFlow dan libary pendukung untuk pemrosesan dan visualisasi data

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import random

print(tf.__version__)

: 

## Download MNIST dataset

In [ ]:
# download MNIST dataset, dan split jadi 2 dataset
# "train" dataset dan "test" dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
# Normalisasikan input gambar agar tiap nilai pixel antara 0 sampai 1
train_images = train_images / 255.0
test_images = test_images / 255.0
print('normalized')

In [ ]:
# tampilkan 25 gambar pertama pada training dataset
plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i], cmap=plt.cm.gray)
  plt.xlabel(train_labels[i])
plt.show()

## Train "TensorFlow" model

Mulai lakukan training model

In [ ]:
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
  keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Dropout(0.25),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Berikut tampilan struktur model nya

In [ ]:
model.summary()

## Evaluate "TensorFlow" model
Evaluasi model menggunakan gambar pada dataset "test"

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

In [ ]:
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

predictions = model.predict(test_images)

prediction_digits = np.argmax(predictions, axis=1)

plt.figure(figsize=(18, 18))
for i in range(100):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  image_index = random.randint(0, len(prediction_digits))
  plt.imshow(test_images[image_index], cmap=plt.cm.gray)
  ax.xaxis.label.set_color(get_label_color(prediction_digits[image_index],\
                                           test_labels[image_index]))
  plt.xlabel('Predicted: %d' % prediction_digits[image_index])
plt.show()

## Convert the "TensorFlow" model to "TensorFlow Lite"

Convert model awal TensorFlow menjadi model TensorFlow Lite agar ukurannya menjadi kecil

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Di convert ulang lagi menggunakan teknik quantization
agar bisa mendapatkan ukuran yang lebih kecil lagi

In [ ]:
# Re-convert using "quantization"
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

## Evaluate "TensorFlow Lite" model

evaluasi model lagi setelah di convert ke TensorFlow Lite untuk mengecek nilai drop accuracy nya

In [ ]:
# evaluasi TF Lite model pake dataset "test"
def evaluate_tflite_model(tflite_model):
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_tensor_index = interpreter.get_input_details()[0]["index"]
  output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

  # lakukan prediksi pake dataset "test"
  prediction_digits = []
  for test_image in test_images:
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_tensor_index, test_image)
    interpreter.invoke()
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy

# nilai akurasi model sebelumnya
float_accuracy = evaluate_tflite_model(tflite_float_model)
print('Float model accuracy = %.4f' % float_accuracy)

# nilai akurasi model setelah di "quantization"
quantized_accuracy = evaluate_tflite_model(tflite_quantized_model)
print('Quantized model accuracy = %.4f' % quantized_accuracy)
print('Accuracy drop = %.4f' % (float_accuracy - quantized_accuracy))


## Download model

In [ ]:
f = open('mnist.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

from google.colab import files
files.download('mnist.tflite')

print('`mnist.tflite` downloaded')